In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")

In [ ]:
nCellsGuide = read.csv("./../TextFiles/NoOfCellsPerGuide.csv")
nCellsGuide$cellRatio = nCellsGuide$nCellsScreen / nCellsGuide$nCellsPool
nCellsGuide$GuideType = "CONTROL"
nCellsGuide[nCellsGuide$type == "Gene", "GuideType"] = "KO Guide"

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)

ggplot(nCellsGuide, aes(x=cellRatio, color=GuideType, fill=GuideType)) +
    #geom_histogram(aes(y=..density..), position="identity", alpha=0.2, binwidth = 0.02)+
    geom_density(alpha=0.1)+
    labs(x="(# of cells in the screen) / (# of cells in the initial pool) ", y = "Density")+
    theme_bw()+geom_vline(xintercept = 0.085, color="blue")

In [ ]:
options(repr.plot.width=7, repr.plot.height=4)
pdf("./../SupplementaryFigures/S_2Z.pdf", height=4, width=7)
ggplot(nCellsGuide, aes(cellRatio, colour = GuideType)) +
  stat_ecdf(geom = "step")+theme_minimal()+
labs(
     x="(# of guides in the screen) / (# of guides in the initial pool)", 
     y = "CDF")+theme(axis.text = element_text(size=15),
              axis.title =  element_text(size=16))+xlim(0, 1)+
geom_segment(aes(x = 0, y = 0.05, xend = 0.08587, yend = 0.05), color="blue")+
geom_segment(aes(x = 0.08587, y = 0, xend = 0.08587, yend = 0.05), color="blue")

dev.off()
#geom_hline(yintercept = 0.05, color="blue")

In [ ]:
k = nCellsGuide[nCellsGuide$GuideType == "CONTROL","cellRatio"]
d_fun <- ecdf (k)
d_fun(0.08587)

In [ ]:
nCellsKOGuides = nCellsGuide[nCellsGuide$type == "Gene",]

In [ ]:
for(i in 1:nrow(nCellsKOGuides)){
    nCellsKOGuides[i,"Pval"] = d_fun(nCellsKOGuides[i,"cellRatio"])
}

nCellsKOGuides$Gene = sapply(nCellsKOGuides$Guide, 
                             function(x){strsplit(x,"_")[[1]][1]})

In [ ]:
write.csv(nCellsKOGuides, "GuideDepletion.csv", row.names=FALSE)

In [ ]:
nCellsKOGuidesDepleted = nCellsKOGuides[nCellsKOGuides$Pval < 0.05,]

In [ ]:
write.csv(nCellsKOGuidesDepleted, "GuideDepletion_ControlGuidesBackground.csv", row.names=FALSE)

In [ ]:
hh <- table(nCellsKOGuidesDepleted$Gene)

In [ ]:
hh = hh[order(hh)]
hh = hh[hh>1]

In [ ]:
data.frame(GuideName=names(hh), NumberOfGuides=hh)